In [33]:
import torch
# assert torch.__version__ == "2.8.0+cu128", (f"Torch version is {torch.__version__} instead of 2.8.0+cu128")
assert torch.cuda.is_available and torch.cuda.device_count() == 1, "GPU not enabled"
import numpy as np
# assert np.__version__ == "2.2.0", (f"Numpy version is {np.__version__} instead of 2.2.0")


In [34]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TRANSFORMERS_NO_FLAX"] = "1"
os.environ["TRITON_PTXAS_PATH"] = "/usr/local/cuda/bin/ptxas"

In [35]:
import sys
print(sys.executable)

/home/jianing/anaconda3/envs/kag2/bin/python


In [36]:
import gc
import time
import warnings

import pandas as pd
import polars as pl
import numpy as np

import torch
# import kaggle_evaluation.aimo_3_inference_server

pd.set_option('display.max_colwidth', None)
start_time = time.time()
cutoff_time = start_time + (4 * 60 + 53) * 60

In [37]:
from vllm import LLM, SamplingParams
warnings.simplefilter('ignore')

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

llm_model_pth = '/mnt/d/Qwen3-1.7B'

MAX_NUM_SEQS = 2
MAX_MODEL_LEN = 32768

llm = LLM(
    llm_model_pth,
    dtype="bfloat16",                # The data type for the model weights and activations
    max_num_seqs=MAX_NUM_SEQS,   # Maximum number of sequences per iteration. Default is 256
    max_model_len=MAX_MODEL_LEN, # Model context length
    trust_remote_code=False,      # Trust remote code (e.g., from HuggingFace) when downloading the model and tokenizer
    tensor_parallel_size=1,      # The number of GPUs to use for distributed execution with tensor parallelism
    gpu_memory_utilization=0.9, # The ratio (between 0 and 1) of GPU memory to reserve for the model
    seed=391,
    enable_prefix_caching=True,
)

INFO 11-27 23:04:32 [utils.py:253] non-default args: {'dtype': 'bfloat16', 'seed': 391, 'max_model_len': 32768, 'enable_prefix_caching': True, 'max_num_seqs': 2, 'disable_log_stats': True, 'model': '/mnt/d/Qwen3-1.7B'}
INFO 11-27 23:04:33 [model.py:631] Resolved architecture: Qwen3ForCausalLM
INFO 11-27 23:04:33 [model.py:1745] Using max model len 32768
INFO 11-27 23:04:33 [scheduler.py:216] Chunked prefill is enabled with max_num_batched_tokens=8192.
(EngineCore_DP0 pid=103695) INFO 11-27 23:04:33 [core.py:93] Initializing a V1 LLM engine (v0.11.2) with config: model='/mnt/d/Qwen3-1.7B', speculative_config=None, tokenizer='/mnt/d/Qwen3-1.7B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dty

(EngineCore_DP0 pid=103695) Process EngineCore_DP0:
(EngineCore_DP0 pid=103695) Traceback (most recent call last):
(EngineCore_DP0 pid=103695)   File "/home/jianing/anaconda3/envs/kag2/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
(EngineCore_DP0 pid=103695)     self.run()
(EngineCore_DP0 pid=103695)   File "/home/jianing/anaconda3/envs/kag2/lib/python3.12/multiprocessing/process.py", line 108, in run
(EngineCore_DP0 pid=103695)     self._target(*self._args, **self._kwargs)
(EngineCore_DP0 pid=103695)   File "/home/jianing/anaconda3/envs/kag2/lib/python3.12/site-packages/vllm/v1/engine/core.py", line 846, in run_engine_core
(EngineCore_DP0 pid=103695)     raise e
(EngineCore_DP0 pid=103695)   File "/home/jianing/anaconda3/envs/kag2/lib/python3.12/site-packages/vllm/v1/engine/core.py", line 833, in run_engine_core
(EngineCore_DP0 pid=103695)     engine_core = EngineCoreProc(*args, **kwargs)
(EngineCore_DP0 pid=103695)                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

RuntimeError: Engine core initialization failed. See root cause above. Failed core proc(s): {}

In [ ]:
import vllm
print(vllm.__version__)

0.11.2


In [ ]:
tokenizer = llm.get_tokenizer()

In [ ]:
import re
import keyword


def extract_boxed_text(text):
    pattern = r'oxed{(.*?)}'
    matches = re.findall(pattern, text)
    if not matches:
        return ""
    for match in matches[::-1]:
        if match != "":
            return match
    return ""


from collections import Counter
import random
def select_answer(answers):
    counter = Counter()
    for answer in answers:
        try:
            if int(answer) == float(answer):
                counter[int(answer)] += 1 + random.random() / 1_000
        except:
            pass
    if not counter:
        return 3
    _, answer = sorted([(v,k) for k,v in counter.items()], reverse=True)[0]
    return answer%10000

In [ ]:
def create_starter_messages(question, index):
    options = []
    for _ in range(3):
        options.append(
            [
                {"role": "system", "content": "You are a helpful and harmless assistant. You are Qwen developed by Alibaba. You should think step-by-step."},
                {"role": "user", "content": question + ' Return final answer within \boxed{}, after taking modulo 1000.'},
            ]
        )
    for _ in range(2):
        options.append(
            [
                {"role": "system", "content": "You are a the most powerful math expert. Please solve the problems with deep resoning. You are careful and always recheck your conduction. You will never give answer directly until you have enough confidence. You should think step-by-step. Return final answer within \\boxed{}, after taking modulo 1000."},
                {"role": "user", "content": question},
            ]
        )
    for _ in range(1):    
        options.append(
            [
                {"role": "system", "content": "You are a helpful and harmless math assistant. You should think step-by-step and you are good at reverse thinking to recheck your answer and fix all possible mistakes. After you get your final answer, take modulo 1000, and return the final answer within \\boxed{}."},
                {"role": "user", "content": question},
            ],
        )

    options.append(
        [
            {
                "role": "system",
                "content": (
                    "You are a mathematical genius. "
                    "Explain each step clearly and verify for consistency. "
                    "Give your final solution in \\boxed{} with the result mod 1000."
                )
            },
            {
                "role": "user",
                "content": question
            }
        ]
    )

    # 11
    options.append(
        [
            {
                "role": "system",
                "content": (
                    "You are a highly accurate math assistant. "
                    "Pay special attention to possible pitfalls. "
                    "Your final answer must be in \\boxed{}, and taken modulo 1000."
                )
            },
            {
                "role": "user",
                "content": question
            }
        ]
    )
    return options[index%len(options)]

In [ ]:
from math import ceil

def predict_for_question(question: str) -> int:
    question_start_time = time.time()
    if time.time() > cutoff_time:
        return 3
    selected_questions_only = True
    # if selected_questions_only and not os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    #     # if "Triangle" not in question:
    #     return 210
    print(question)
    
    if time.time() > cutoff_times[-1]:
        print(time.time(), cutoff_times[-1])
        print("SPEEDUP!!!")
        num_of_iters = 2
        num_of_seqs=6
    else:
        num_of_iters = 3
        num_of_seqs=6


    messages = [create_starter_messages(question, index) for index in range(num_of_seqs)]
    list_of_texts = [
        tokenizer.apply_chat_template(
            conversation=message,
            tokenize=False,
            add_generation_prompt=True
        )
        for message in messages
    ]
    old = list_of_texts
    new = list_of_texts
    res = []

    for k in range(num_of_iters):
        # if k == 1:
        sampling_params = SamplingParams(
            temperature=0.7,              # randomness of the sampling
            min_p=0.05,
            top_p=0.9,
            skip_special_tokens=True,     # Whether to skip special tokens in the output
            max_tokens=32768 ,
            # stop=["</think>"]
        )
        outputs = llm.generate(old,  sampling_params=sampling_params)
        print(outputs[0].outputs[0].text)
        new = []

        new_old = []
        for i in range(len(old)):
            if extract_boxed_text(outputs[i].outputs[0].text):
                res.append(old[i] + outputs[i].outputs[0].text)
            else:
                new_old.append(old[i] + outputs[i].outputs[0].text)       

        if num_of_iters == 3 and k == 1 and len(new_old) >= 7:
            curr_num = min(7, len(new_old))
            
            new_old = random.sample(new_old, curr_num)
        print(list(extract_boxed_text(x) for x in res))
        if k == 0:
            repeats = 2
        elif k == 1:
            repeats = 1
        for i in range(len(new_old)):
            for _ in range(repeats):
                new.append(new_old[i])
        old = new

        print(list(extract_boxed_text(x) for x in res))

        if len(res) >= 8:
            numbers = []
            for r in res:
                try:
                    num = int(extract_boxed_text(r))
                    numbers.append(num)
                except ValueError:
                    pass
            # Если удалось собрать 7+ числовых ответов
            if len(numbers) >= 8:
                counter = Counter(numbers)
                total = len(numbers)
                
                for num, cnt in counter.items():
                    if cnt >= ceil(total * 0.8):
                        cutoff_times.pop()
                        question_end_time = time.time()
                        print(num)
                        print("TIME FOR QUESTION: ", question_end_time - question_start_time)
                        print("PRE RESULT!!!")
                        return num
            
    answer = select_answer(list(extract_boxed_text(x) for x in res))
    print(answer)
    cutoff_times.pop()
    question_end_time = time.time()
    print("TIME FOR QUESTION: ", question_end_time - question_start_time)
    return answer

In [ ]:
# Replace this function with your inference code.
# The function should return a single integer between 0 and 999, inclusive.
# Each prediction (except the very first) must be returned within 30 minutes of the question being provided.
def predict(id_: pl.DataFrame, question: pl.DataFrame) -> pl.DataFrame:
    id_ = id_
    # print("------")
    # print(id_)
    question_str = question
    # print(question_str)
    answer = predict_for_question(question_str)
    # print("------\n\n")
    return pl.DataFrame({'id': id_, 'answer': answer})

In [ ]:
problems = pd.read_csv('/home/jianing/nlp/aimo3/ai-mathematical-olympiad-progress-prize-3/test.csv'
)

In [ ]:
cutoff_times = [int(x) for x in np.linspace(cutoff_time, time.time() + 500, 50)]

In [ ]:
submissions = []
for index, problem_row in problems.iterrows():
    
    id_value = problem_row['id']
    question_value = problem_row['problem']
    
    # Pass a single-column Polars Series instead of a DataFrame created from a dictionary.
    # This might allow the internal .item(0) to correctly interpret the index 0 as the row index.
    id_pl_df = pl.Series([id_value]).to_frame()
    question_pl_df = pl.Series([question_value]).to_frame()
    
    # The predict function expects two positional arguments, so we pass the two resulting DataFrames
    submission_result = predict(id_pl_df, question_pl_df)
    
    submissions.append(submission_result)

shape: (1, 1)
┌────────────────┐
│                │
│ ---            │
│ str            │
╞════════════════╡
│ What is $1-1$? │
└────────────────┘


Processed prompts: 100%|██████████| 6/6 [00:17<00:00,  2.99s/it, est. speed input: 18.48 toks/s, output: 150.23 toks/s]


['2', '2']
['2', '2']


Processed prompts: 100%|██████████| 8/8 [00:00<00:00, 86.10it/s, est. speed input: 15603.29 toks/s, output: 108.34 toks/s]


['2', '2']
['2', '2']


...

['376']
['376']


Processed prompts: 100%|██████████| 10/10 [00:00<00:00, 97.41it/s, est. speed input: 17882.38 toks/s, output: 98.46 toks/s]


['376']
['376']


Processed prompts: 100%|██████████| 7/7 [00:00<00:00, 121.50it/s, est. speed input: 20285.89 toks/s, output: 122.72 toks/s]

['376']
['376']
376
TIME FOR QUESTION:  309.16942524909973


In [ ]:
submissions

[shape: (1, 2)
 ┌────────┬────────┐
 │ id     ┆ answer │
 │ ---    ┆ ---    │
 │ str    ┆ i32    │
 ╞════════╪════════╡
 │ 000aaa ┆ 2      │
 └────────┴────────┘,
 shape: (1, 2)
 ┌────────┬────────┐
 │ id     ┆ answer │
 │ ---    ┆ ---    │
 │ str    ┆ i32    │
 ╞════════╪════════╡
 │ 111bbb ┆ 376    │
 └────────┴────────┘,
 shape: (1, 2)
 ┌────────┬────────┐
 │ id     ┆ answer │
 │ ---    ┆ ---    │
 │ str    ┆ i32    │
 ╞════════╪════════╡
 │ 222ccc ┆ 376    │
 └────────┴────────┘]

In [ ]:
df = pl.concat(submissions)
df.write_csv('/home/jianing/nlp/aimo3/ai-mathematical-olympiad-progress-prize-3/submission.csv')